In [4]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.


In [5]:

import cv2
# do not delete the data folder
# the data folder will store the images for training the model 

converting images to grayscale

generating datasets


In [6]:

def generate_dataset():
    face_classifier =cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # Scaling factor = 1.3
        # Minimum neighbor = 5
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            face_cropped = img[y:y+h, x:x+w]
        return face_cropped

    cap = cv2.VideoCapture(0)
    # change the id number for next user , by default for 1st user it is set to 1
    id = 1
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (400,400))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            # (50,50) is the origin point from where text is to be written
            # Font scale = 1
            # Thickness = 2
            cv2.imshow("Cropped Face", face)
            # 13 is the ASCII key value for ENTER
            if cv2.waitKey(1) == 13 or int(img_id) == 400:
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed.......")

# Call the function to start dataset generation
generate_dataset()


KeyboardInterrupt: 

training the classifier and saving

In [ ]:
import numpy as np
from PIL import Image
import os
import cv2

In [ ]:
def train_classifier(data_dir):
    #joining images(f) to the data (eg.user1.2)
    path =[os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces=[]
    ids =[]
    
    for image in path:
        img = Image.open(image).convert('L');
        imageNp =np.array(img,'uint8')
        id =int(os.path.split(image)[1].split(".")[1])
        # D:\4th semester\Face_recognizer_project\data\user.1.2

        faces.append(imageNp)
        ids.append(id)
    ids=np.array(ids)

    #trainig and saving classifier
    clf= cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

Detecting the faces and naming if it is already installed


In [ ]:
def draw_boundary(img,classifier,scaleFactor,minNeighbours,color,text,clf):
    gray_image =cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbours)

    coords =[]

    for(x,y,w,h) in features:
        cv2.rectangle(img,(x,y),(x+w,y+h),color,4)
        id,pred = clf.predict(gray_image[y:y+h,x:x+w])
        confidence =int(100*(1-pred/300))

        if confidence>60:
            if id==1:
                cv2.putText(img,"Ravi",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
                # for other id continue it with elif 
        else:
            cv2.putText(img,"UNKNOWN..🤔",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,266),1,cv2.LINE_AA) 
        
        coords =[x,y,w,h]
    return coords 

def recognize(img,clf,faceCascade):
    coords = draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
    return img

faceCascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf= cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture =cv2.VideoCapture(0)
while True:
    ret,img = video_capture.read()
    img =recognize(img,clf,faceCascade)
    cv2.imshow("face detected ...👍",img)

    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows()